In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris

## 1) 載入資料集

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target

print('X 資料: ',X.shape)
print('y 資料: ',y.shape)

## 2) 切割訓練集與測試集

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25 , random_state=40)

print('訓練資料: ',X_train.shape)
print('測試資料: ',X_test.shape)

訓練資料:  (112, 4)
測試資料:  (38, 4)


## KMeans

In [ ]:
from sklearn.cluster import KMeans

kmeansModel = KMeans(n_clusters=3, random_state=46)
clusters_pred = kmeansModel.fit_predict(X_train)

In [ ]:
print(f'True label: {y_test[0]}')
print(kmeansModel.predict(np.array([X_test[0]])))

## 輸出 ONNX

In [17]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(kmeansModel, initial_types=initial_type)
with open("./model/kmeans_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

載入並測試。

In [1]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy as np
sess = rt.InferenceSession("./model/kmeans_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
x = np.array([[6.7, 3.1, 4.4, 1.4]], dtype=np.float32)
pred_onx = sess.run([label_name], {input_name: x})[0]
print(pred_onx)

[1]
